# Comparação entre Experimentos de Noisy Neighbors

Este notebook demonstra como comparar diferentes experimentos de noisy neighbors para identificar padrões, diferenças e semelhanças entre configurações experimentais.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from scipy import stats

# Ajustar o path para importar os módulos do pipeline
sys.path.append(os.path.abspath('../..'))

# Importar módulos necessários do pipeline
from pipeline.data_processing.consolidation import load_experiment_data, select_tenants
from pipeline.data_processing.time_normalization import normalize_time
from pipeline.data_processing.aggregation import aggregate_by_time
from pipeline.analysis.advanced_analysis import calculate_covariance_matrix, calculate_entropy_metrics
from pipeline.analysis.phase_analysis import compare_phases
from pipeline.analysis.anomaly_detection import detect_anomalies_ensemble
from pipeline.visualization.plots import plot_comparison, create_heatmap

# Configurar visualização
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

## 1. Carregar Dados de Múltiplos Experimentos

Primeiro, vamos carregar dados de diferentes experimentos para comparação.

In [ ]:
# Definir caminhos para diferentes experimentos
experiment_paths = [
    '../../demo-data/demo-experiment-3-rounds/',
    # Adicionar caminhos para outros experimentos conforme necessário
]

# Carregar dados de cada experimento
experiments = {}
for i, path in enumerate(experiment_paths):
    try:
        metrics_data, exp_info = load_experiment_data(path)
        
        # Usar o nome do experimento se disponível, caso contrário usar um ID
        exp_name = exp_info.get('name', f'experiment_{i+1}')
        
        # Armazenar dados e metadados
        experiments[exp_name] = {
            'metrics': metrics_data,
            'info': exp_info
        }
        
        print(f"Carregado experimento: {exp_name}")
        print(f"  - Métricas: {list(metrics_data.keys())}")
        print(f"  - Tenants: {exp_info.get('tenants', [])}")
        print(f"  - Fases: {exp_info.get('phases', [])}")
        print()
        
    except Exception as e:
        print(f"Erro ao carregar experimento em {path}: {str(e)}")

## 2. Preparar Dados para Comparação

Vamos preparar os dados para comparar experimentos, normalizando o tempo e selecionando métricas de interesse.

In [ ]:
# Métricas de interesse para comparação
metrics_of_interest = ['cpu_usage', 'memory_usage']

# Preprocessar dados para cada experimento
for exp_name, exp_data in experiments.items():
    # Filtrar métricas de interesse
    selected_metrics = {k: v for k, v in exp_data['metrics'].items() if k in metrics_of_interest}
    
    # Normalizar tempo para cada métrica
    normalized_metrics = {}
    for metric_name, df in selected_metrics.items():
        # Normalizar tempo
        norm_df = normalize_time(df, exp_data['info'])
        
        # Adicionar coluna de tempo decorrido em minutos
        norm_df['elapsed_minutes'] = (norm_df['datetime'] - norm_df['datetime'].min()).dt.total_seconds() / 60
        
        # Agregar dados por minuto para uniformizar comparações
        agg_df = aggregate_by_time(norm_df, time_column='elapsed_minutes', freq='1min')
        
        normalized_metrics[metric_name] = agg_df
    
    # Atualizar dados do experimento
    exp_data['processed_metrics'] = normalized_metrics

# Verificar uma amostra dos dados processados
if experiments:
    exp_name = list(experiments.keys())[0]
    for metric in metrics_of_interest:
        if metric in experiments[exp_name]['processed_metrics']:
            print(f"Amostra de {metric} para {exp_name}:")
            display(experiments[exp_name]['processed_metrics'][metric].head())
            print()

## 3. Comparação de Estatísticas Básicas entre Experimentos

Vamos comparar estatísticas básicas (média, mediana, desvio padrão, etc.) entre experimentos.

In [ ]:
# Função para calcular estatísticas resumidas
def calculate_statistics(df, metric_column='value', group_by=None):
    if group_by:
        stats_df = df.groupby(group_by)[metric_column].agg([
            ('média', 'mean'),
            ('mediana', 'median'),
            ('desvio_padrao', 'std'),
            ('mínimo', 'min'),
            ('máximo', 'max'),
            ('contagem', 'count')
        ])
    else:
        stats_df = pd.DataFrame({
            'média': [df[metric_column].mean()],
            'mediana': [df[metric_column].median()],
            'desvio_padrao': [df[metric_column].std()],
            'mínimo': [df[metric_column].min()],
            'máximo': [df[metric_column].max()],
            'contagem': [df[metric_column].count()]
        })
    
    return stats_df

In [ ]:
# Comparar estatísticas básicas entre experimentos
for metric in metrics_of_interest:
    print(f"Comparação de estatísticas para {metric}:\n")
    
    # Criar DataFrame para comparação
    all_stats = []
    
    # Calcular estatísticas para cada experimento
    for exp_name, exp_data in experiments.items():
        if metric in exp_data['processed_metrics']:
            # Estatísticas globais
            df = exp_data['processed_metrics'][metric]
            stats_df = calculate_statistics(df)
            stats_df['experimento'] = exp_name
            stats_df['grupo'] = 'global'
            all_stats.append(stats_df)
            
            # Estatísticas por tenant
            tenant_stats = calculate_statistics(df, group_by=['tenant'])
            tenant_stats = tenant_stats.reset_index()
            tenant_stats['experimento'] = exp_name
            tenant_stats['grupo'] = tenant_stats['tenant']
            all_stats.append(tenant_stats)
            
            # Estatísticas por fase
            phase_stats = calculate_statistics(df, group_by=['phase'])
            phase_stats = phase_stats.reset_index()
            phase_stats['experimento'] = exp_name
            phase_stats['grupo'] = phase_stats['phase']
            all_stats.append(phase_stats)
    
    # Consolidar estatísticas
    if all_stats:
        combined_stats = pd.concat(all_stats, ignore_index=True)
        
        # Pivotar para facilitar comparação entre experimentos
        pivot_stats = combined_stats.pivot_table(
            index=['grupo'],
            columns=['experimento'],
            values=['média', 'mediana', 'desvio_padrao', 'máximo']
        )
        
        # Exibir estatísticas comparativas
        display(pivot_stats)
        print("\n" + "-"*80 + "\n")

## 4. Comparação de Distribuições entre Experimentos

Vamos visualizar e comparar as distribuições de métricas entre experimentos.

In [ ]:
# Função para comparar distribuições de métricas entre experimentos
def compare_distributions(experiments, metric, tenant=None, phase=None):
    plt.figure(figsize=(14, 6))
    
    # Organizar dados para comparação
    data_to_plot = []
    labels = []
    
    for exp_name, exp_data in experiments.items():
        if metric in exp_data['processed_metrics']:
            df = exp_data['processed_metrics'][metric]
            
            # Filtrar por tenant ou fase se especificados
            if tenant:
                df = df[df['tenant'] == tenant]
            if phase:
                df = df[df['phase'] == phase]
            
            if not df.empty:
                data_to_plot.append(df['value'])
                labels.append(exp_name)
    
    if not data_to_plot:
        return "Dados insuficientes para comparação"
    
    # Plotar distribuições (boxplot)
    plt.subplot(1, 2, 1)
    plt.boxplot(data_to_plot, labels=labels)
    plt.title(f'Boxplot de {metric}' + 
             (f' para {tenant}' if tenant else '') + 
             (f' na fase {phase}' if phase else ''))
    plt.ylabel('Valor')
    plt.xticks(rotation=45)
    plt.grid(True, alpha=0.3)
    
    # Plotar distribuições (violin plot)
    plt.subplot(1, 2, 2)
    for i, data in enumerate(data_to_plot):
        plt.violinplot(data, positions=[i+1])
    plt.title(f'Violin Plot de {metric}' + 
             (f' para {tenant}' if tenant else '') + 
             (f' na fase {phase}' if phase else ''))
    plt.ylabel('Valor')
    plt.xticks(range(1, len(labels)+1), labels, rotation=45)
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Executar teste estatístico para comparar distribuições
    if len(data_to_plot) >= 2:
        print("Comparações estatísticas:")
        for i in range(len(data_to_plot)):
            for j in range(i+1, len(data_to_plot)):
                # Kolmogorov-Smirnov test
                ks_stat, ks_pval = stats.ks_2samp(data_to_plot[i], data_to_plot[j])
                
                # Mann-Whitney U test
                u_stat, u_pval = stats.mannwhitneyu(data_to_plot[i], data_to_plot[j], 
                                                   alternative='two-sided')
                
                print(f"\n{labels[i]} vs {labels[j]}:")
                print(f"  Kolmogorov-Smirnov test: statistic={ks_stat:.4f}, p-value={ks_pval:.4f}")
                print(f"  Mann-Whitney U test: statistic={u_stat:.4f}, p-value={u_pval:.4f}")
                
                if ks_pval < 0.05:
                    print("  Conclusão K-S: As distribuições são significativamente diferentes")
                else:
                    print("  Conclusão K-S: Não há evidência suficiente de que as distribuições são diferentes")

In [ ]:
# Comparar distribuições para cada métrica
for metric in metrics_of_interest:
    print(f"\n{'-'*40} Comparação de {metric} {'-'*40}\n")
    
    # Comparação global
    print("\nComparação global:")
    compare_distributions(experiments, metric)
    
    # Comparação para tenant específico (se houver dados suficientes)
    common_tenants = []
    for exp_data in experiments.values():
        if 'tenants' in exp_data['info']:
            if not common_tenants:
                common_tenants = exp_data['info']['tenants']
            else:
                common_tenants = [t for t in common_tenants if t in exp_data['info']['tenants']]
    
    if common_tenants:
        print(f"\nComparação para tenant {common_tenants[0]}:")
        compare_distributions(experiments, metric, tenant=common_tenants[0])
    
    # Comparação para fase específica (se houver dados suficientes)
    common_phases = []
    for exp_data in experiments.values():
        if 'phases' in exp_data['info']:
            if not common_phases:
                common_phases = exp_data['info']['phases']
            else:
                common_phases = [p for p in common_phases if p in exp_data['info']['phases']]
    
    if common_phases:
        print(f"\nComparação para fase {common_phases[0]}:")
        compare_distributions(experiments, metric, phase=common_phases[0])

## 5. Comparação de Anomalias entre Experimentos

Vamos comparar a prevalência e os tipos de anomalias detectadas em diferentes experimentos.

In [ ]:
# Detectar anomalias em cada experimento
anomaly_results = {}

for exp_name, exp_data in experiments.items():
    anomaly_results[exp_name] = {}
    
    for metric in metrics_of_interest:
        if metric in exp_data['processed_metrics']:
            df = exp_data['processed_metrics'][metric]
            
            # Detectar anomalias
            df_with_anomalies, _ = detect_anomalies_ensemble(
                df, 
                metric_column='value',
                time_column='elapsed_minutes',
                contamination=0.05,
                group_by=['tenant']
            )
            
            # Armazenar resultados
            anomaly_results[exp_name][metric] = df_with_anomalies

# Resumir resultados de anomalias por experimento
anomaly_summary = []

for exp_name, metrics_results in anomaly_results.items():
    for metric, df in metrics_results.items():
        # Estatísticas globais
        total_points = len(df)
        anomaly_if_count = df['is_anomaly_if'].sum()
        anomaly_lof_count = df['is_anomaly_lof'].sum()
        anomaly_ensemble_count = df['is_anomaly'].sum() if 'is_anomaly' in df.columns else 0
        change_point_count = df['is_change_point'].sum() if 'is_change_point' in df.columns else 0
        
        anomaly_summary.append({
            'experimento': exp_name,
            'métrica': metric,
            'grupo': 'global',
            'total_pontos': total_points,
            'anomalias_if': anomaly_if_count,
            'anomalias_lof': anomaly_lof_count,
            'anomalias_ensemble': anomaly_ensemble_count,
            'pontos_mudanca': change_point_count,
            'pct_anomalias': (anomaly_ensemble_count / total_points * 100) if total_points > 0 else 0
        })
        
        # Estatísticas por tenant
        for tenant in df['tenant'].unique():
            tenant_df = df[df['tenant'] == tenant]
            total_points = len(tenant_df)
            anomaly_if_count = tenant_df['is_anomaly_if'].sum()
            anomaly_lof_count = tenant_df['is_anomaly_lof'].sum()
            anomaly_ensemble_count = tenant_df['is_anomaly'].sum() if 'is_anomaly' in tenant_df.columns else 0
            change_point_count = tenant_df['is_change_point'].sum() if 'is_change_point' in tenant_df.columns else 0
            
            anomaly_summary.append({
                'experimento': exp_name,
                'métrica': metric,
                'grupo': tenant,
                'total_pontos': total_points,
                'anomalias_if': anomaly_if_count,
                'anomalias_lof': anomaly_lof_count,
                'anomalias_ensemble': anomaly_ensemble_count,
                'pontos_mudanca': change_point_count,
                'pct_anomalias': (anomaly_ensemble_count / total_points * 100) if total_points > 0 else 0
            })

# Converter para DataFrame e exibir
anomaly_summary_df = pd.DataFrame(anomaly_summary)

# Exibir resumo global
print("Resumo global de anomalias por experimento e métrica:\n")
display(anomaly_summary_df[anomaly_summary_df['grupo'] == 'global'])

# Criar visualização comparativa
if not anomaly_summary_df.empty:
    plt.figure(figsize=(14, 6))
    
    # Filtrar apenas estatísticas globais
    global_stats = anomaly_summary_df[anomaly_summary_df['grupo'] == 'global']
    
    # Criar gráfico de barras para porcentagem de anomalias por experimento e métrica
    sns.barplot(x='experimento', y='pct_anomalias', hue='métrica', data=global_stats)
    plt.title('Porcentagem de Anomalias por Experimento')
    plt.ylabel('% de Pontos Anômalos')
    plt.xlabel('Experimento')
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

## 6. Comparação de Covariância e Correlação entre Experimentos

Vamos analisar e comparar as relações de covariância e correlação entre tenants em diferentes experimentos.

In [ ]:
# Calcular e comparar matrizes de covariância
for exp_name, exp_data in experiments.items():
    print(f"\n{'-'*20} Matrizes de Covariância/Correlação para {exp_name} {'-'*20}\n")
    
    for metric in metrics_of_interest:
        if metric in exp_data['processed_metrics']:
            print(f"\nMétrica: {metric}")
            
            # Organizar dados para cálculo de covariância
            metrics_dict = {metric: exp_data['processed_metrics'][metric]}
            
            try:
                # Calcular matrizes
                cov_matrix, corr_matrix = calculate_covariance_matrix(
                    metrics_dict,
                    tenants=None,  # Usar todos os tenants disponíveis
                    phase=None,    # Usar todas as fases
                    round_name=exp_data['processed_metrics'][metric]['round'].iloc[0]  # Usar primeiro round disponível
                )
                
                # Exibir matrizes
                print("\nMatriz de Correlação:")
                display(corr_matrix)
                
                # Visualizar matriz de correlação como heatmap
                plt.figure(figsize=(10, 8))
                sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', 
                           vmin=-1, vmax=1, fmt='.2f', linewidths=0.5)
                plt.title(f'Matriz de Correlação - {metric} ({exp_name})')
                plt.tight_layout()
                plt.show()
                
            except Exception as e:
                print(f"Erro ao calcular matrizes: {str(e)}")

## 7. Visualização Comparativa de Séries Temporais

Vamos criar visualizações para comparar diretamente as séries temporais de diferentes experimentos.

In [ ]:
# Função para comparar séries temporais entre experimentos
def compare_time_series(experiments, metric, tenant=None, phase=None):
    plt.figure(figsize=(14, 8))
    
    # Organizar dados para plotagem
    for exp_name, exp_data in experiments.items():
        if metric in exp_data['processed_metrics']:
            df = exp_data['processed_metrics'][metric]
            
            # Filtrar por tenant ou fase se especificados
            if tenant:
                df = df[df['tenant'] == tenant]
            if phase:
                df = df[df['phase'] == phase]
            
            # Verificar se temos dados após filtragem
            if not df.empty:
                plt.plot(df['elapsed_minutes'], df['value'], label=exp_name, alpha=0.7)
    
    plt.title(f'Comparação de {metric}' + 
             (f' para {tenant}' if tenant else '') + 
             (f' na fase {phase}' if phase else ''))
    plt.xlabel('Tempo Decorrido (minutos)')
    plt.ylabel('Valor')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
# Comparar séries temporais para cada métrica
for metric in metrics_of_interest:
    print(f"\n{'-'*40} Comparação Temporal de {metric} {'-'*40}\n")
    
    # Comparação global
    print("\nComparação global:")
    compare_time_series(experiments, metric)
    
    # Comparação para tenant específico (se houver dados comuns)
    if common_tenants:
        print(f"\nComparação para tenant {common_tenants[0]}:")
        compare_time_series(experiments, metric, tenant=common_tenants[0])
    
    # Comparação para fase específica (se houver dados comuns)
    if common_phases:
        print(f"\nComparação para fase {common_phases[0]}:")
        compare_time_series(experiments, metric, phase=common_phases[0])

## 8. Análise de Entropia e Informação Mútua entre Experimentos

Vamos calcular e comparar métricas de entropia cruzada e informação mútua entre tenants para diferentes experimentos.

In [ ]:
# Calcular métricas de entropia para cada experimento
entropy_results = {}

for exp_name, exp_data in experiments.items():
    entropy_results[exp_name] = {}
    
    for metric in metrics_of_interest:
        if metric in exp_data['processed_metrics']:
            df = exp_data['processed_metrics'][metric]
            
            # Calcular métricas de entropia
            try:
                entropy_df = calculate_entropy_metrics(
                    df,
                    tenants=None,  # Usar todos os tenants disponíveis
                    phase=None,    # Usar todas as fases
                    metric_column='value'
                )
                
                entropy_results[exp_name][metric] = entropy_df
            except Exception as e:
                print(f"Erro ao calcular entropia para {metric} em {exp_name}: {str(e)}")
                entropy_results[exp_name][metric] = pd.DataFrame()

# Exibir e comparar resultados
for metric in metrics_of_interest:
    print(f"\n{'-'*40} Comparação de Entropia e Informação Mútua - {metric} {'-'*40}\n")
    
    results_to_compare = []
    
    for exp_name, metrics_results in entropy_results.items():
        if metric in metrics_results and not metrics_results[metric].empty:
            # Adicionar coluna de experimento
            df = metrics_results[metric].copy()
            df['experimento'] = exp_name
            results_to_compare.append(df)
    
    if results_to_compare:
        # Consolidar resultados
        combined_entropy = pd.concat(results_to_compare, ignore_index=True)
        display(combined_entropy)
        
        # Visualizar informação mútua entre experimentos
        plt.figure(figsize=(12, 6))
        sns.barplot(x='tenant1', y='mutual_information', hue='experimento', data=combined_entropy)
        plt.title(f'Comparação de Informação Mútua entre Tenants - {metric}')
        plt.xlabel('Tenant 1')
        plt.ylabel('Informação Mútua')
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()

## 9. Análise Dimensional e Clustering de Experimentos

Vamos usar técnicas de redução de dimensionalidade e clustering para identificar padrões e agrupar experimentos similares.

In [ ]:
# Função para extrair características dos experimentos
def extract_experiment_features(experiments, metric):
    features = []
    
    for exp_name, exp_data in experiments.items():
        if metric in exp_data['processed_metrics']:
            df = exp_data['processed_metrics'][metric]
            
            # Extrair características estatísticas
            for tenant in df['tenant'].unique():
                tenant_data = df[df['tenant'] == tenant]
                
                # Calcular estatísticas
                mean_val = tenant_data['value'].mean()
                std_val = tenant_data['value'].std()
                max_val = tenant_data['value'].max()
                min_val = tenant_data['value'].min()
                median_val = tenant_data['value'].median()
                skew_val = tenant_data['value'].skew()
                kurtosis_val = tenant_data['value'].kurtosis()
                
                # Adicionar ao conjunto de características
                features.append({
                    'experimento': exp_name,
                    'métrica': metric,
                    'tenant': tenant,
                    'média': mean_val,
                    'desvio_padrão': std_val,
                    'máximo': max_val,
                    'mínimo': min_val,
                    'mediana': median_val,
                    'assimetria': skew_val,
                    'curtose': kurtosis_val
                })
    
    return pd.DataFrame(features)

In [ ]:
# Extrair características para análise dimensional
for metric in metrics_of_interest:
    print(f"\n{'-'*40} Análise Dimensional para {metric} {'-'*40}\n")
    
    # Extrair características
    features_df = extract_experiment_features(experiments, metric)
    
    if features_df.empty:
        print(f"Dados insuficientes para análise de {metric}")
        continue
    
    # Exibir características extraídas
    print("Características extraídas:")
    display(features_df)
    
    # Preparar dados para PCA e t-SNE
    numeric_features = ['média', 'desvio_padrão', 'máximo', 'mínimo', 'mediana', 'assimetria', 'curtose']
    X = features_df[numeric_features].values
    
    # Normalizar dados
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # PCA
    if X_scaled.shape[0] > 1 and X_scaled.shape[1] > 1:
        pca = PCA(n_components=2)
        X_pca = pca.fit_transform(X_scaled)
        
        # t-SNE (se houver dados suficientes)
        if X_scaled.shape[0] >= 5:
            tsne = TSNE(n_components=2, random_state=42)
            X_tsne = tsne.fit_transform(X_scaled)
            
            # Visualizar resultados
            plt.figure(figsize=(16, 6))
            
            # PCA plot
            plt.subplot(1, 2, 1)
            for exp in features_df['experimento'].unique():
                mask = features_df['experimento'] == exp
                plt.scatter(X_pca[mask, 0], X_pca[mask, 1], label=exp, alpha=0.8, s=50)
            
            for i, txt in enumerate(features_df['tenant']):
                plt.annotate(txt, (X_pca[i, 0], X_pca[i, 1]), fontsize=9)
                
            plt.title(f'PCA - {metric}')
            plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
            plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')
            plt.grid(True, alpha=0.3)
            plt.legend()
            
            # t-SNE plot
            plt.subplot(1, 2, 2)
            for exp in features_df['experimento'].unique():
                mask = features_df['experimento'] == exp
                plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1], label=exp, alpha=0.8, s=50)
            
            for i, txt in enumerate(features_df['tenant']):
                plt.annotate(txt, (X_tsne[i, 0], X_tsne[i, 1]), fontsize=9)
                
            plt.title(f't-SNE - {metric}')
            plt.xlabel('Dimensão 1')
            plt.ylabel('Dimensão 2')
            plt.grid(True, alpha=0.3)
            plt.legend()
            
            plt.tight_layout()
            plt.show()
        else:
            # Apenas PCA se não houver dados suficientes para t-SNE
            plt.figure(figsize=(8, 6))
            for exp in features_df['experimento'].unique():
                mask = features_df['experimento'] == exp
                plt.scatter(X_pca[mask, 0], X_pca[mask, 1], label=exp, alpha=0.8, s=50)
            
            for i, txt in enumerate(features_df['tenant']):
                plt.annotate(txt, (X_pca[i, 0], X_pca[i, 1]), fontsize=9)
                
            plt.title(f'PCA - {metric}')
            plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.2f}%)')
            plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.2f}%)')
            plt.grid(True, alpha=0.3)
            plt.legend()
            plt.tight_layout()
            plt.show()

## 10. Conclusões e Próximos Passos

Este notebook demonstrou como comparar diferentes experimentos de noisy neighbors para identificar padrões, diferenças e semelhanças entre eles. As principais análises realizadas incluíram:

1. **Comparação de estatísticas básicas**: média, mediana, desvio padrão, etc.
2. **Comparação de distribuições**: usando boxplots, violin plots e testes estatísticos
3. **Comparação de anomalias**: análise da prevalência e tipos de anomalias entre experimentos
4. **Análise de covariância e correlação**: comparação das relações entre tenants
5. **Visualização comparativa de séries temporais**: sobreposição direta das métricas
6. **Análise de entropia e informação mútua**: comparação de interdependências entre tenants
7. **Análise dimensional**: uso de PCA e t-SNE para visualizar e agrupar características

### Próximos Passos:

- Expandir as comparações para incluir mais experimentos com diferentes configurações
- Desenvolver métricas de similaridade entre experimentos
- Implementar análises estatísticas mais avançadas para quantificar diferenças
- Integrar com o módulo de geração de relatórios para documentação automática
- Utilizar técnicas de aprendizado de máquina para prever resultados de novos experimentos com base nos existentes
- Criar um framework para avaliar o impacto de diferentes estratégias de mitigação de interferência entre tenants